In [10]:
import requests
from PIL import Image
import json

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [2]:
# model_name
model_name = "llava-hf/llava-1.5-7b-hf"

# load model
model = LlavaForConditionalGeneration.from_pretrained(
    model_name,
     torch_dtype=torch.bfloat16,
     attn_implementation="flash_attention_2",
     device_map="auto",
)
model.eval()

# load processor
processor = AutoProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [6]:
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "What are these?"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

In [7]:
prompt

'USER: <image>\nWhat are these? ASSISTANT:'

In [16]:
#prompt = "USER: <image>\nWhat are these? ASSISTANT:"
#prompt = "USER: What are these?\n<image> ASSISTANT:"

In [8]:
image_file = "http://images.cocodataset.org/val2017/000000039769.jpg"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(images=raw_image, text=prompt, return_tensors='pt').to("cuda")

output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

ER:  
What are these? ASSISTANT: These are two cats lying on a pink couch.


In [11]:
folder_path = '/root/vlm-compositionality/data/raw/sugarcrepe/'

# add attribute
with open (folder_path+'add_att.json') as f:
    add_attribute = json.load(f)

# add object
with open (folder_path+'add_obj.json') as f:
    add_object = json.load(f)

# replace attribute
with open (folder_path+'replace_att.json') as f:
    replace_attribute = json.load(f)

# replace object
with open (folder_path+'replace_obj.json') as f:
    replace_object = json.load(f)

# replace relation
with open (folder_path+'replace_rel.json') as f:
    replace_relation = json.load(f)

# swap attribute
with open (folder_path+'swap_att.json') as f:
    swap_attribute = json.load(f)

# swap object
with open (folder_path+'swap_obj.json') as f:
    swap_object = json.load(f)

# collate together
dataset = {
    'add_attribute': add_attribute, 'add_object': add_object, 'replace_attribute': replace_attribute,
    'replace_object': replace_object, 'replace_relation': replace_relation,
    'swap_attribute': swap_attribute, 'swap_object': swap_object,
}

In [12]:
img_folder = '/root/vlm-compositionality/data/raw/coco_val_2017/'

In [72]:
id = '67'
img_file = swap_object[id]['filename']
caption = swap_object[id]['caption']
negative_caption = swap_object[id]['negative_caption']
swap_object[id]

{'filename': '000000021503.jpg',
 'caption': 'Crackers coated with spread, sitting on a plate, ready to eat.',
 'negative_caption': 'Spread coated with crackers, sitting on a plate, ready to eat.'}

In [31]:
#chat = [

  #{"role": "user", "content": "Hello, how are you?"},

  #{"role": "assistant", "content": "I'm doing great. How can I help you today?"},

  #{"role": "user", "content": "I'd like to show off how chat templating works!"},

#]

In [49]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: Men wash a motorcycle while girls look on. Caption 2: Girls wash a motorcycle while men look on."},
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Caption 2"}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: The stop sign is behind the fence instead of on the street. Caption 2: The fence is behind the stop sign instead of on the street."},
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Caption 1"}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": ("Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: {} Caption 2: {}").format(negative_caption, caption)},
        ]
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text

'USER: <image>\nChoose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: Men wash a motorcycle while girls look on. Caption 2: Girls wash a motorcycle while men look on. ASSISTANT: Caption 2 USER: <image>\nChoose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: The stop sign is behind the fence instead of on the street. Caption 2: The fence is behind the stop sign instead of on the street. ASSISTANT: Caption 1 USER: <image>\nChoose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: A girl with a ball facing a man with a racquet. Caption 2: A man with a ball facing a girl with a racquet. ASSISTANT:'

In [50]:
images = [
    img_folder+'000000177934.jpg',
    img_folder+'000000011122.jpg',
    img_folder+img_file,
]
    
raw_images = [Image.open(image) for image in images]
inputs = processor(images=raw_images, text=text, return_tensors='pt').to("cuda")

In [51]:
output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

ER:  
Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: Men wash a motorcycle while girls look on. Caption 2: Girls wash a motorcycle while men look on. ASSISTANT: Caption 2 USER:  
Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: The stop sign is behind the fence instead of on the street. Caption 2: The fence is behind the stop sign instead of on the street. ASSISTANT: Caption 1 USER:  
Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: A girl with a ball facing a man with a racquet. Caption 2: A man with a ball facing a girl with a racquet. ASSISTANT: Caption 1 USER: A man with a ball facing a girl with a racquet.


In [61]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": ("Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: {} Caption 2: {}").format(negative_caption, caption)},
        ]
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text

'USER: <image>\nChoose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: A couple men pat an elephant as a woman watches. Caption 2: A woman pats an elephant as a couple men watch. ASSISTANT:'

In [62]:
images = [
    img_folder+img_file,
]
    
raw_images = [Image.open(image) for image in images]
inputs = processor(images=raw_images, text=text, return_tensors='pt').to("cuda")

output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

ER:  
Choose and return the correct caption for the image from the following 2 captions. Generate no other text. Caption 1: A couple men pat an elephant as a woman watches. Caption 2: A woman pats an elephant as a couple men watch. ASSISTANT: Caption 1: A couple men pat an elephant as a woman watches.


In [73]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": ("Select the correct caption and only the caption number for the given image. The captions are as follows : Caption 1: {} Caption 2: {}").format(negative_caption, caption)},
        ]
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text

'USER: <image>\nSelect the correct caption and only the caption number for the given image. The captions are as follows : Caption 1: Spread coated with crackers, sitting on a plate, ready to eat. Caption 2: Crackers coated with spread, sitting on a plate, ready to eat. ASSISTANT:'

In [74]:
images = [
    img_folder+img_file,
]
    
raw_images = [Image.open(image) for image in images]
inputs = processor(images=raw_images, text=text, return_tensors='pt').to("cuda")

output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

ER:  
Select the correct caption and only the caption number for the given image. The captions are as follows : Caption 1: Spread coated with crackers, sitting on a plate, ready to eat. Caption 2: Crackers coated with spread, sitting on a plate, ready to eat. ASSISTANT: Caption 1
